In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA

from imblearn.under_sampling import NeighbourhoodCleaningRule
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from skbayes.rvm_ard_models import RVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn import grid_search
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTETomek
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import (EditedNearestNeighbours,RepeatedEditedNearestNeighbours)

In [ ]:
names = ["Nearest Neighbors","RVM", "RBF SVM", "Gaussian Process",
         "Random Forest", "Neural Net"]

classifiers = [
    KNeighborsClassifier(p=2),
    RVC(kernel='rbf'),
    SVC(kernel='rbf'),
    GaussianProcessClassifier(1.0 * RBF(0.5), warm_start=True),
    RandomForestClassifier(max_depth=3, n_estimators=100),
    MLPClassifier(alpha=0.5)]

In [ ]:
wb=pd.ExcelFile(r"C:\Users\cvck2\Desktop\akis_data\akis_cleared.xlsx")
sheet1=pd.read_excel(r"C:\Users\cvck2\Desktop\akis_data\akis_cleared.xlsx",sheetname=0)
print sheet1.columns
df=pd.DataFrame(sheet1["Sp_up_t-5"])
df2=pd.DataFrame(sheet1["Sp_down_t-5"])
df3=pd.DataFrame(sheet1["Vol_up_t-5"])
df4=pd.DataFrame(sheet1["Vol_down_t-5"])
df5=pd.DataFrame(sheet1["Occ_up_t-5"])
df6=pd.DataFrame(sheet1["Occ_down_t-5"])
dfall=[df,df2,df3,df4,df5,df6]

X=pd.concat(dfall,axis=1)
response=pd.DataFrame(sheet1["Accident"])
respomse=np.ravel(response)
r_binary=label_binarize(respomse,classes=[0,1])
y=np.ravel(r_binary)
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_sample(X, y)

In [ ]:
parameters_svm=[{'gamma': np.linspace(1e-4,1,num=100),'C': [0.1, 0.3, 0.5, 1, 10, 100]}]
parameters_rvm=[{'gamma': np.linspace(1e-4,1,num=100)}]
parameters_gp=[{'length_scale': np.linspace(1e-4,1,num=100)}]
parameters_knn=[{'n_neighbors':np.linspace(10,200,50)}]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled,y_resampled, test_size=0.33, random_state=0)

In [ ]:
plt.figure(figsize=(10, 10))
for name, clf in zip(names, classifiers):
    if name=="RVM":
        CV_clf = grid_search.GridSearchCV(clf, parameters_rvm, cv=10)
        
        clf=CV_clf.fit(X_train, y_train)
        print(clf.best_estimator_)
    elif name=="RBF SVM":
        CV_clf = grid_search.GridSearchCV(clf, parameters_svm, cv=10)
        
        clf=CV_clf.fit(X_train, y_train)
        print(clf.best_estimator_)
    elif name=="Nearest Neighbors":
        CV_clf=grid_search.GridSearchCV(clf, parameters_knn, cv=10)
        CV_clf2= CV_clf.estimator
        clf=CV_clf2.fit(X_train, y_train)
        print(CV_clf2)
    else: clf.fit(X_train,y_train)
    
   
        # Plot the decision boundary. For that, we will assign a color to each
        # point in the mesh [x_min, x_max]x[y_min, y_max].
    if hasattr(clf, "decision_function"):
            Z = clf.decision_function(X_test)
    else:
            Z = clf.predict_proba(X_test)[:, 1]
    y_pred=clf.predict(X_test)
    fpr,tpr, _ =roc_curve(y_test,Z)
    roc_auc=auc(fpr,tpr)
    print 'Confusion Matrix' + repr(name) 
    print confusion_matrix(y_test,y_pred)
    print 'Classification Report' +repr(name)
    print classification_report(y_test,y_pred)
    lw = 2
    plt.plot(fpr, tpr, "-", label="%s'ROC curve area = %0.2f)' " % (name,roc_auc), lw=2 )
plt.plot([0, 1], [0, 1], 'k--',lw=3, label="Random Guess")
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve of classifiers using 5-minute traffic data from Athens')
plt.legend(loc='best')
plt.savefig('ROC_5_akis_nc.png')
plt.show()